In [ ]:
#imports
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd
import os
import tensorflow as tf

mpl.rcParams["font.family"] = "Times New Roman"
mpl.rcParams["font.size"] = 12
mpl.rcParams["text.usetex"] = True

import common as my

# !pwd && ls -ltr
file_path = "outputs/"
outputs = os.listdir(file_path)
files = my.get_files(outputs, "demo", ".txt")

# Chapter 2
### 2.1.1 Reproduction

In [ ]:
df = pd.read_csv(file_path+files[100], delimiter=",", header=0)
df

In [ ]:
fig = plt.figure(figsize=(6,5), dpi=150)
colors = {df.columns[5]: "b", df.columns[6]:"b" ,df.columns[7]: "r"}
linestypes = {df.columns[5]:"-", df.columns[6]: "-.", df.columns[7]: "-"}
labels = {df.columns[5]: "$L^1(\lambda_{[0,1]^d}; \mathrm{R})$", 
          df.columns[6]:"$L^2(\lambda_{[0,1]^d}; \mathrm{R})$", 
          df.columns[7]:"$L^{\infty}(\lambda_{[0,1]^d}; \mathrm{R})$"}
xvalues = df[df.columns[0]]

for i in [5, 6,7]:
    it = df.columns[i]
    plt.plot(xvalues, df[it], label=labels[it], 
             color=colors[it],linestyle=linestypes[it]
            )

plt.xlim([0,3e5])
plt.ylim([0,1.2])
plt.ylabel("Estimated Relative Errors")
plt.xlabel("Number of Iterations")
plt.grid()
plt.legend()
plt.title("Results of Heat Equation with $d=100$ by Deep Learning")
plt.savefig("/Users/michaellea/001 - M.Sc_HPC/55640-Project/Seminar-2/Presentation/Slides/F01.png", bbox_inches='tight')
plt.show()

- Plot the Predictions

In [ ]:
def gen_grid_world(up,low, grid_size, dim='2d'):
    xx = np.linspace(up,low,grid_size)
    yy = np.linspace(up,low,grid_size)

    if dim == '2d':
        X,Y = np.meshgrid(xx, yy)
        return np.column_stack((X.ravel(), Y.ravel()))
    elif dim == '3d':
        zz = np.linspace(0,1,100)
        X, Y, Z = np.meshgrid(xx, yy, zz)
        return np.column_stack((X.ravel(), Y.ravel(), Z.ravel()))
    else:
        raise ValueError("ExpectValue: '2d', '3d'")

def phi(x):
    return tf.reduce_sum(x ** 2, axis=1, keepdims=True)
    
def custom_loss(model, x, y, training):
    y_ = model(x, training=training)
    return tf.reduce_mean((y_- y)**2)

In [ ]:
T = 1
d = 2

Grid_World = gen_grid_world(0,1,200, dim='2d')
u_ref = phi(Grid_World) + 2.*T*d

In [ ]:
model = tf.keras.models.load_model(f'best_models/best_model_{d}d',custom_objects={'custom_loss': custom_loss})

In [ ]:
# Create a 2D plots
fig = plt.figure(figsize=(10,3.5), dpi=200)


ax = fig.add_subplot(121)
scatter = ax.scatter(Grid_World[:,0], Grid_World[:,1], c=u_ref, cmap='jet', marker='o',alpha=1, s=3)
cbar = fig.colorbar(scatter, ax=ax, label='$u(1, x, y)$')

# Set labels and title
ax.set_xlabel('X-axis')
ax.set_ylabel('Y-axis')
ax.set_title('Solution')

plt.xlim([0,1])
plt.ylim([0,1])

ax2 = fig.add_subplot(122)
cbar = fig.colorbar(scatter, ax=ax2, label='$u(1, x, y)$')
scatter = ax2.scatter(Grid_World[:,0], Grid_World[:,1], alpha=1, cmap='jet', marker='o', c = model(Grid_World, training=False),s=3)

# Set labels and title
ax2.set_xlabel('X-axis')
ax2.set_ylabel('Y-axis')
ax2.set_title('Prediction')

plt.xlim([0,1])
plt.ylim([0,1])

plt.savefig(f"/Users/michaellea/001 - M.Sc_HPC/55640-Project/Seminar-2/Presentation/Slides/F0{d}.png", bbox_inches='tight')

plt.show()

In [ ]:
T = 1
d = 3

Grid_World = gen_grid_world(0,1,200, dim=f'{d}d')
u_ref = phi(Grid_World) + 2.*T*d

In [ ]:
model = tf.keras.models.load_model(f'best_models/best_model_{d}d',custom_objects={'custom_loss': custom_loss})

In [ ]:
# Create a 3D plots
fig = plt.figure(figsize=(15,5), dpi=200)


ax = fig.add_subplot(121, projection='3d')
scatter = ax.scatter(Grid_World[:,0], Grid_World[:,1], Grid_World[:,2], c=u_ref, cmap='jet', marker='o',alpha=.5, s=2)
cbar = fig.colorbar(scatter, ax=ax, label='$u(1, x, y)$')
plt.subplots_adjust(right=0.8)

# Set labels and title
ax.set_xlabel('X-axis')
ax.set_ylabel('Y-axis')
ax.set_title('Solution')

plt.xlim([0,1])
plt.ylim([0,1])

ax2 = fig.add_subplot(122, projection='3d')
cbar = fig.colorbar(scatter, ax=ax2, label='$u(1, x, y)$')
scatter = ax2.scatter(Grid_World[:,0], Grid_World[:,1], Grid_World[:,2], alpha=.5, cmap='jet', marker='o', c = model(Grid_World, training=False),s=2)

# Set labels and title
ax2.set_xlabel('X-axis')
ax2.set_ylabel('Y-axis')
ax2.set_title('Prediction')

plt.xlim([0,1])
plt.ylim([0,1])

plt.savefig(f"/Users/michaellea/001 - M.Sc_HPC/55640-Project/Seminar-2/Presentation/Slides/F0{d}.png", bbox_inches='tight')

plt.show()

In [ ]:
T = 1
d = 3

Grid_World = gen_grid_world(0,1,100, dim=f'{d}d')
d = 100
u_ref = phi(Grid_World) + 2.*T*d

model = tf.keras.models.load_model(f'best_models/best_model_{d}d',custom_objects={'custom_loss': custom_loss})

In [ ]:
temp = np.zeros((1000000,100))
for i, _ in enumerate(temp):
    temp[i,0:3] = Grid_World[i]
Grid_World = temp

In [ ]:
# Create a 3D plots
fig = plt.figure(figsize=(15,5), dpi=200)


ax = fig.add_subplot(121, projection='3d')
scatter = ax.scatter(Grid_World[:,0], Grid_World[:,1], Grid_World[:,2], c=u_ref, cmap='jet', marker='o',alpha=.5, s=2)
cbar = fig.colorbar(scatter, ax=ax, label='$u(1, x, y)$')
plt.subplots_adjust(right=0.8)

# Set labels and title
ax.set_xlabel('X-axis')
ax.set_ylabel('Y-axis')
ax.set_title('Solution')

plt.xlim([0,1])
plt.ylim([0,1])

ax2 = fig.add_subplot(122, projection='3d')
cbar = fig.colorbar(scatter, ax=ax2, label='$u(1, x, y)$')
scatter = ax2.scatter(Grid_World[:,0], Grid_World[:,1], Grid_World[:,2], alpha=.5, cmap='jet', marker='o', c = model(Grid_World, training=False),s=2)

# Set labels and title
ax2.set_xlabel('X-axis')
ax2.set_ylabel('Y-axis')
ax2.set_title('Prediction')

plt.xlim([0,1])
plt.ylim([0,1])

plt.savefig(f"/Users/michaellea/001 - M.Sc_HPC/55640-Project/Seminar-2/Presentation/Slides/F0{d}.png", bbox_inches='tight')

plt.show()

### 2.1.2 Detial I've Found

### 2.1.3 Numerical Methods DIFF (C)

In [ ]:
path_2d =  "../../Heat2D/difference2D.dat"
path_3d =  "../../Heat3D/difference3D.dat"

In [ ]:
df2d = pd.read_csv(path_2d, delimiter=" ")
df3d = pd.read_csv(path_3d, delimiter=" ")

df2d.head(3),df3d.head(3)

In [ ]:
list(df2d["step"])[-1]

In [ ]:
fig = plt.figure(figsize=(6,5), dpi=300)
colors = {"2d": "b", "3d":"r"}
linestypes = {"2d":"-", "3d": "-"}
labels = {"2d": "dim-$2$", "3d":"dim-$3$"}
xvalues = df2d[df2d.columns[0]]

for i, df in enumerate([df2d, df3d]):
    plt.semilogy(df[df.columns[0]], df["diff"], label=labels[f"{i+2}d"], 
             color=colors[f"{i+2}d"],linestyle=linestypes[f"{i+2}d"]
            )

# plt.semilogy
# plt.xlim([0,3e5])
# plt.ylim([0,1.2])
plt.ylabel("Differences")
plt.xlabel("Number of Iterations")
plt.grid()
plt.legend()

plt.yticks([1e1, 1e-2, 1e-4, 1e-6, 1e-8])
plt.xticks([0,500,1000,1400, list(df2d["step"])[-1], 2100, 2400, list(df3d["step"])[-1]])
plt.title("Results of Heat Equation by Finite Differences")
plt.savefig("/Users/michaellea/001 - M.Sc_HPC/55640-Project/Seminar-2/Presentation/Slides/F0FDM.png", bbox_inches='tight')
plt.show()